In [1]:
import constellate
import json
import pandas as pd
import pickle
pd.set_option("display.max_columns", None)
import os
import csv

Constellate: use and download of datasets is covered by the Terms & Conditions of Use: https://constellate.org/terms-and-conditions/


In [2]:
dataset = constellate.dataset_reader("../data/large_files/jstor_v2023.jsonl")

In [3]:
type(dataset)

generator

In [4]:
data_columns = ['unigramCount', 'bigramCount', 'trigramCount', 'abstract', 'fullText']


In [5]:
path_trigrams = "../data/large_files/trigrams_raw/"
#os.makedirs(path_trigrams)
path_unigrams = "../data/large_files/unigrams_raw/"
#os.makedirs(path_unigrams)

In [7]:
%%time
metadata_list = []
dataset = constellate.dataset_reader("../data/large_files/jstor_v2023.jsonl")
for n, doc_dict in enumerate(dataset):
        doc_dict["id_kase"] = n
        trigrams = doc_dict["trigramCount"]
        with open(path_trigrams + "trigrams_{}.json".format(str(n)), "w") as f:
            json.dump(trigrams, f)
        unigrams = doc_dict["unigramCount"]
        with open(path_unigrams + "unigrams_{}.json".format(str(n)), "w") as f:
            json.dump(unigrams, f)
        for col in ['unigramCount', 'bigramCount', 'trigramCount']: # , 'fullText']:
            try:
                del doc_dict[col]
            except:
                pass
        metadata_list.append(doc_dict)

CPU times: user 4min 39s, sys: 32 s, total: 5min 11s
Wall time: 5min 43s


In [8]:
metadata_df = pd.DataFrame(metadata_list)
len(metadata_df)

25000

In [19]:
metadata_df["fullText"].notnull().sum()

625

In [20]:
metadata_df.drop("fullText", axis=1, inplace=True)

In [93]:
metadata_df.to_json("../data/metadata_df.json")

In [94]:
ids_dict = dict(zip(metadata_df["id"], metadata_df["id_kase"]))

In [95]:
with open("../data/large_files/ids_dict.json", "w") as f:
    json.dump(ids_dict, f)